Tests on Ensemble learning methods

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from matplotlib import cm
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
import kagglehub

# ----------------------------------------
# 1. Download and Prepare the Dataset
# ----------------------------------------
path = kagglehub.dataset_download("pkdarabi/bone-break-classification-image-dataset")
path = os.path.join(path, 'Bone Break Classification/Bone Break Classification')

def collect_image_data_paths(directory):
    data = []
    # Scan each class folder
    for class_folder in os.listdir(directory):
        class_path = os.path.join(directory, class_folder)
        print(class_path)
        if os.path.isdir(class_path):
            for split in ['Train', 'Test']:
                split_path = os.path.join(class_path, split)
                if os.path.isdir(split_path):
                    for image_name in os.listdir(split_path):
                        image_path = os.path.join(split_path, image_name)
                        data.append({'path': image_path, 'target': class_folder, 'split': split})
    return data

# Collect the data and create a DataFrame
data = collect_image_data_paths(path)
df = pd.DataFrame(data)
print(df.head())



/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification/Avulsion fracture
/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification/Spiral Fracture
/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification/Impacted fracture
/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification/Hairline Fracture
/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification/Greenstick fracture
/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification/Pathological fracture
/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification/Oblique fracture
/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification/Fracture Dislocation
/k

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 10
img_height = 256
img_width = 256

# Filter the full training DataFrame
train_df_full = df[df['split'] == 'Train']

# Perform a stratified split into training and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(
    train_df_full,
    test_size=0.2,
    random_state=42,
    stratify=train_df_full['target']
)

# Create separate ImageDataGenerators for training, validation, and testing
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='target',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create the validation generator
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='path',
    y_col='target',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create the test generator
test_generator = test_datagen.flow_from_dataframe(
    dataframe=df[df['split'] == 'Test'],
    x_col='path',
    y_col='target',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Print the total number of samples in each generator
print(f'Train samples: {train_generator.samples}')
print(f'Validation samples: {val_generator.samples}')
print(f'Test samples: {test_generator.samples}')

# Print the number of images per class for each set using value_counts()
print("\nTrain samples per class:")
print(train_df['target'].value_counts())

print("\nValidation samples per class:")
print(val_df['target'].value_counts())

print("\nTest samples per class:")
test_df = df[df['split'] == 'Test']
print(test_df['target'].value_counts())

Found 791 validated image filenames belonging to 10 classes.
Found 198 validated image filenames belonging to 10 classes.
Found 140 validated image filenames belonging to 10 classes.
Train samples: 791
Validation samples: 198
Test samples: 140

Train samples per class:
target
Fracture Dislocation     110
Comminuted fracture      107
Pathological fracture     93
Avulsion fracture         87
Greenstick fracture       85
Hairline Fracture         81
Impacted fracture         60
Spiral Fracture           59
Oblique fracture          55
Longitudinal fracture     54
Name: count, dtype: int64

Validation samples per class:
target
Fracture Dislocation     27
Comminuted fracture      27
Pathological fracture    23
Avulsion fracture        22
Greenstick fracture      21
Hairline Fracture        20
Spiral Fracture          15
Impacted fracture        15
Longitudinal fracture    14
Oblique fracture         14
Name: count, dtype: int64

Test samples per class:
target
Fracture Dislocation     19
Pat

In [3]:
import os
import numpy as np
import cv2
from matplotlib import cm
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from sklearn.model_selection import train_test_split

# --- Assume df, train_df, val_df, test_df already defined as before ---

batch_size  = 10
img_height  = 256
img_width   = 256
num_classes = train_df['target'].nunique()

# --- 1. Preprocessing functions ---

def orig_preprocess(x):
    # x is a uint8 array in [0,255], shape (H,W,3)
    return preprocess_input(x)

def gray_preprocess(x):
    # Convert to true grayscale, then back to 3‑channel
    gray = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
    gray3 = np.stack([gray, gray, gray], axis=-1)
    return preprocess_input(gray3)

def edge_preprocess(x):
    # x may be float32 in [0,1] (if you used rescale=1./255)
    # or float32 in [0,255] (if not). Detect and convert to uint8 0–255:
    if x.dtype != np.uint8:
        if x.max() <= 1.0:
            x_uint8 = (x * 255).astype(np.uint8)
        else:
            x_uint8 = x.astype(np.uint8)
    else:
        x_uint8 = x

    # Now true grayscale + Canny
    gray = cv2.cvtColor(x_uint8, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)

    # Stack to 3 channels and apply EfficientNet preprocessing
    e3 = np.stack([edges, edges, edges], axis=-1)
    return preprocess_input(e3.astype(np.uint8))

def jet_preprocess(x):
    gray = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
    norm = (gray - gray.min()) / (gray.max() - gray.min() + 1e-7)
    colored = cm.get_cmap('jet')(norm)[..., :3]   # floats [0,1]
    colored255 = (colored * 255).astype(np.uint8)
    return preprocess_input(colored255)

# --- 2. Generator factory ---

def make_generator(df, preprocess_fn, shuffle):
    return ImageDataGenerator(preprocessing_function=preprocess_fn) \
        .flow_from_dataframe(
            dataframe=df,
            x_col='path',
            y_col='target',
            target_size=(img_height, img_width),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=shuffle
        )

# Training generators (shuffle=True)
orig_train = make_generator(train_df, orig_preprocess, shuffle=True)
gray_train = make_generator(train_df, gray_preprocess, shuffle=True)
edge_train = ImageDataGenerator(preprocessing_function=edge_preprocess) \
    .flow_from_dataframe(train_df, x_col='path', y_col='target',
                         target_size=(256,256), batch_size=10, class_mode='categorical')
jet_train  = make_generator(train_df, jet_preprocess,  shuffle=True)

# Validation generators (shuffle=False for consistent ordering)
orig_val = make_generator(val_df, orig_preprocess, shuffle=False)
gray_val = make_generator(val_df, gray_preprocess, shuffle=False)
edge_val   = ImageDataGenerator(preprocessing_function=edge_preprocess) \
    .flow_from_dataframe(val_df,   x_col='path', y_col='target',
                         target_size=(256,256), batch_size=10, class_mode='categorical',
                         shuffle=False)
jet_val  = make_generator(val_df, jet_preprocess,  shuffle=False)

# --- 3. Model factory (EfficientNetB3 → GAP → 256 → Dropout → 520 → Dropout → softmax) ---

def build_branch_model():
    inp = layers.Input((img_height, img_width, 3))
    base = EfficientNetB3(
        weights='imagenet',
        include_top=False,
        input_tensor=inp
    )
    base.trainable = False

    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(520, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)

    m = models.Model(inp, out)
    m.compile(
        optimizer=optimizers.RMSprop(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy'
        ]
    )
    return m

# --- 4. Instantiate branches ---

model_orig = build_branch_model()
model_gray = build_branch_model()
model_edge = build_branch_model()
model_jet  = build_branch_model()

# --- 5. Train each branch (you should see your grayscale branch recover ~45% val‑acc) ---

model_orig.fit(orig_train, validation_data=orig_val, epochs=100)
model_gray.fit(gray_train, validation_data=gray_val, epochs=100)
model_edge.fit(edge_train, validation_data=edge_val, epochs=100)
model_jet.fit(jet_train,   validation_data=jet_val, epochs=100)

# --- 6. Ensemble by averaging softmax probabilities ---

import math

def ensemble_val_accuracy(models, val_generator):
    steps = math.ceil(val_generator.samples / val_generator.batch_size)
    # collect each model’s predictions (shape: [N, num_classes])
    prob_list = [m.predict(val_generator, steps=steps) for m in models]
    avg_probs = np.mean(prob_list, axis=0)
    y_true = val_generator.classes
    y_pred = np.argmax(avg_probs, axis=1)
    return np.mean(y_pred == y_true)

val_acc = ensemble_val_accuracy(
    [model_orig, model_gray, model_edge, model_jet],
    orig_val   # any val_* generator will do, since ordering is the same
)
print(f"Ensemble validation accuracy: {val_acc:.4f}")


Found 791 validated image filenames belonging to 10 classes.
Found 791 validated image filenames belonging to 10 classes.
Found 791 validated image filenames belonging to 10 classes.
Found 791 validated image filenames belonging to 10 classes.
Found 198 validated image filenames belonging to 10 classes.
Found 198 validated image filenames belonging to 10 classes.
Found 198 validated image filenames belonging to 10 classes.
Found 198 validated image filenames belonging to 10 classes.
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 99s 654ms/step - accuracy: 0.1410 - loss: 2.3521 - val_accuracy: 0.1566 - val_loss: 2.2318
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.1736 - loss: 2.2375 - val_accuracy: 0.2172 - val_loss: 2.1841
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.1986 - loss: 2.1954 - val_accuracy: 0.2323 - val_loss: 2.1458
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.2602 - loss: 2.0814 - val_accuracy: 0.2828 - val_loss: 2.1026
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.2928 - loss: 2.0758 - val_accuracy: 0.2727 - val_loss: 2.0710
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.3273 - loss: 1.9737 - val_accuracy: 0.3182 - val_loss: 2.0240
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.3557 - loss: 1.9410 - val_accuracy: 0.3131 - val_loss: 1.9975
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.3573 - loss: 1.8977 - val_accuracy: 

<ipython-input-3-d69c1f3b1c1c>:53: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colored = cm.get_cmap('jet')(norm)[..., :3]   # floats [0,1]


80/80 ━━━━━━━━━━━━━━━━━━━━ 63s 385ms/step - accuracy: 0.1184 - loss: 2.3096 - val_accuracy: 0.2020 - val_loss: 2.2252
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.1653 - loss: 2.2424 - val_accuracy: 0.2424 - val_loss: 2.1783
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.2058 - loss: 2.1776 - val_accuracy: 0.2626 - val_loss: 2.1358
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.2591 - loss: 2.1080 - val_accuracy: 0.3131 - val_loss: 2.0966
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.2964 - loss: 2.0057 - val_accuracy: 0.3182 - val_loss: 2.0577
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.2929 - loss: 2.0241 - val_accuracy: 0.3333 - val_loss: 2.0249
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.3285 - loss: 1.9418 - val_accuracy: 0.3434 - val_loss: 1.9820
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.3567 - loss: 1.9008 - val_accuracy: 0.3636 - val